In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
# import cv2
from PIL import Image
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
# from tensorflow.keras.layers.normalization import BatchNormalization
import os
import seaborn as sns
from keras.applications import DenseNet121
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# train_data_path = '/content/drive/My Drive/Colab Notebooks/QR_Code_Imgs/Training'
# test_data_path = '/content/drive/My Drive/Colab Notebooks/QR_Code_Imgs/Testing'

In [ ]:
train_data_path = 'Training'
test_data_path = 'Testing'

In [ ]:
x_train = []
y_train = []
x_test = []
y_test = []

In [ ]:
# Function to process images in a folder
def process_images(folder_path, images_list, labels_list):
    for class_folder in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_folder)
        if not os.path.isdir(class_path):
            continue

        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            image = Image.open(image_path)

            # Perform any necessary preprocessing steps here
            # For example, resize, convert to grayscale, normalize, etc.
#             image = image.resize(input_shape)

            # Convert the image to a NumPy array
            image_array = np.array(image)

            # Normalize the image data (if needed)
            image_array = image_array / 255.0

            # Append the processed image and its label to the lists
            images_list.append(image_array)
            labels_list.append(class_folder)

In [ ]:
# Process training images
process_images(train_data_path, x_train, y_train)

# Process testing images
process_images(test_data_path, x_test, y_test)

In [ ]:
# Convert the lists to NumPy arrays
x_train = np.array(x_train)
x_test = np.array(x_test)

In [ ]:
# Convert the lists to NumPy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

In [ ]:
# Convert class labels to categorical format using one-hot encoding
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

onehot_encoder = OneHotEncoder(sparse=False)
y_train1 = onehot_encoder.fit_transform(y_train_encoded.reshape(-1, 1))
y_test1 = onehot_encoder.transform(y_test_encoded.reshape(-1, 1))

In [ ]:
x_train.shape, x_test.shape

In [ ]:
y_train1.shape, y_test1.shape

In [ ]:
# Reshape the data by adding an extra dimension
x_train1 = np.expand_dims(x_train, axis=-1)
x_test1 = np.expand_dims(x_test, axis=-1)

In [ ]:
x_train1.shape, x_test1.shape, y_train1.shape, y_test1.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
# Define the shape of the input data
input_shape = (224, 224, 1)

# Define the number of classes
num_classes = 5

# Define the 2D CNN model architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', padding='same', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
# Fully connected
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
# Output layer
model.add(Dense(num_classes, activation='softmax'))

model.summary()

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Fit the model to the training data
history=model.fit(x_train1, y_train1, batch_size=64, epochs=100)

In [ ]:
_, accuracy = model.evaluate(x_test, y_test1, batch_size=64, verbose=0)

In [ ]:
prediction = model.predict(x_test1)

In [ ]:
prediction_NN=np.argmax(prediction, axis=-1)

In [ ]:
prediction_NN

In [ ]:
y_test1_1=np.argmax(y_test1, axis=-1)

In [ ]:
y_test1_1

In [ ]:
#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test1_1, prediction_NN)
print(cm)

In [ ]:
from sklearn.metrics import classification_report

# print(class_labels)
target_names = ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4']
print(classification_report(y_test1_1, prediction_NN, target_names=target_names,digits=3))

In [ ]:
# import matplotlib.pyplot as plt

In [ ]:
# from mlxtend.plotting import plot_confusion_matrix

In [ ]:
# class_dict = {0: 'Class 0',
#               1: 'Class 1',
#               2: 'Class 2',
#               3: 'Class 3',
#               4: 'Class 4'}

# multiclass = np.array([[],
#                        [],
#                        [],
#                        [],
#                        []])

# fig, ax = plot_confusion_matrix(
#     conf_mat=multiclass,
#     class_names=class_dict.values(),show_normed=True,colorbar=True)